<a href="https://colab.research.google.com/github/YKulki/Turbulence-Modelling/blob/main/ANN_Turb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time


In [3]:
datapath='/content/drive/MyDrive/Turbulence_data/data.npy'
dataset=np.load(datapath)
dataset.shape

(1046, 600, 6)

In [4]:
train=dataset[:,0:400,:]
test=dataset[:,401:599,:]
train.shape

(1046, 400, 6)

In [6]:
class FCN(nn.Module):
    "Defines a connected network"
    
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
        
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

                


In [7]:
# train standard neural network to fit training data
torch.manual_seed(123)
model=FCN(3,3,32,3)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
for i in range(100):
    optimizer.zero_grad()
    x_data=train[:,:,0:3]
    x_data=torch.tensor(x_data,requires_grad=True)

    y_data=train[:,:,3:6]
    y_data=torch.tensor(y_data,requires_grad=True)

    # loss due to data
    yh = model(x_data.float())
    loss1 = torch.mean((yh-y_data)**2) # MSE


    #insert values of u,v,p,x,y,t here
    u=y_data[:,:,0]
    v=y_data[:,:,1]
    p=y_data[:,:,2]

    # gradients
    '''
    ux=torch.autograd.grad(yh[:,:,0],x_data[:,:,1])[0]
    uy=torch.autograd.grad(yh[:,:,0],y,torch.ones_like(u))
    uxx=torch.autograd.grad(ux,x,torch.ones_like(ux))
    uyy=torch.autograd.grad(uy,y,torch.ones_like(uy))
    ut=torch.autograd.grad(yh[:,:,0],t,torch.ones_like(u))
    vx=torch.autograd.grad(yh[:,:,1],x,torch.ones_like(v))
    vy=torch.autograd.grad(yh[:,:,1],y,torch.ones_like(v))
    vxx=torch.autograd.grad(vx,x,torch.ones_like(vx))
    vyy=torch.autograd.grad(vy,y,torch.ones_like(vy))
    vt=torch.autograd.grad(yh[:,:,1],t,torch.ones_like(v))
    px=torch.autograd.grad(yh[:,:,2],x,torch.ones_like(p))
    py=torch.autograd.grad(yh[:,:,2],y,torch.ones_like(p))

    #invariant losses

    loss_mc=ux
    loss_NVx=ut+u*ux+v*uy+px-V*(uxx+uyy)
    loss_NVy=vt+u*vx+v*vy+py-V*(vxx+vyy)
'''
    # backpropagation
    loss=loss1
    loss.backward()
    optimizer.step()
               